In [1]:
import os, sys, time, glob, random, argparse
import numpy as np
from copy import deepcopy
import torch
import torch.nn as nn

# XAutoDL 
from xautodl.config_utils import load_config, dict2config, configure2str
from xautodl.datasets import get_datasets, get_nas_search_loaders
from xautodl.procedures import (
    prepare_seed,
    prepare_logger,
    save_checkpoint,
    copy_checkpoint,
    get_optim_scheduler,
)
from xautodl.utils import get_model_infos, obtain_accuracy
from xautodl.log_utils import AverageMeter, time_string, convert_secs2time
from xautodl.models import get_search_spaces

from custom_models import get_cell_based_tiny_net

# NB201
from nas_201_api import NASBench201API as API

2022-09-28 02:13:20.567295: I tensorflow/core/util/util.cc:169] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.


In [2]:
parser = argparse.ArgumentParser("Random search for NAS.")
parser.add_argument("--data_path", type=str, default='../cifar.python', help="The path to dataset")
parser.add_argument("--dataset", type=str, default='cifar10',choices=["cifar10", "cifar100", "ImageNet16-120"], help="Choose between Cifar10/100 and ImageNet-16.")

# channels and number-of-cells
parser.add_argument("--search_space_name", type=str, default='nas-bench-201', help="The search space name.")
parser.add_argument("--config_path", type=str, default='../configs/nas-benchmark/algos/RANDOM.config', help="The path to the configuration.")
parser.add_argument("--max_nodes", type=int, default=4, help="The maximum number of nodes.")
parser.add_argument("--channel", type=int, default=16, help="The number of channels.")
parser.add_argument("--num_cells", type=int, default=1, help="The number of cells in one stage.")
parser.add_argument("--select_num", type=int, default=100, help="The number of selected architectures to evaluate.")
parser.add_argument("--track_running_stats", type=int, default=0, choices=[0, 1], help="Whether use track_running_stats or not in the BN layer.")
# log
parser.add_argument("--workers", type=int, default=4, help="number of data loading workers")
parser.add_argument("--save_dir", type=str, default='../results/220924_single_cell_random/', help="Folder to save checkpoints and log.")
# parser.add_argument("--arch_nas_dataset", type=str, default='../NAS-Bench-201-v1_1-096897.pth', help="The path to load the architecture dataset (tiny-nas-benchmark).")
parser.add_argument("--arch_nas_dataset", type=str, default=None, help="The path to load the architecture dataset (tiny-nas-benchmark).")
parser.add_argument("--print_freq", type=int, default=100, help="print frequency (default: 200)")
parser.add_argument("--rand_seed", type=int, default=None, help="manual seed")
args = parser.parse_args(args=[])
if args.rand_seed is None or args.rand_seed < 0:
    args.rand_seed = random.randint(1, 100000)

    
print(args.rand_seed)
print(args)
xargs=args

7350
Namespace(arch_nas_dataset=None, channel=16, config_path='../configs/nas-benchmark/algos/RANDOM.config', data_path='../cifar.python', dataset='cifar10', max_nodes=4, num_cells=1, print_freq=100, rand_seed=7350, save_dir='../results/220924_single_cell_random/', search_space_name='nas-bench-201', select_num=100, track_running_stats=0, workers=4)


In [3]:
assert torch.cuda.is_available(), "CUDA is not available."
torch.backends.cudnn.enabled = True
torch.backends.cudnn.benchmark = False
torch.backends.cudnn.deterministic = True
torch.set_num_threads(xargs.workers)
prepare_seed(xargs.rand_seed)
logger = prepare_logger(args)

Main Function with logger : Logger(dir=../results/220924_single_cell_random, use-tf=False, writer=None)
Arguments : -------------------------------
arch_nas_dataset : None
channel          : 16
config_path      : ../configs/nas-benchmark/algos/RANDOM.config
data_path        : ../cifar.python
dataset          : cifar10
max_nodes        : 4
num_cells        : 1
print_freq       : 100
rand_seed        : 7350
save_dir         : ../results/220924_single_cell_random/
search_space_name : nas-bench-201
select_num       : 100
track_running_stats : 0
workers          : 4
Python  Version  : 3.7.13 (default, Mar 29 2022, 02:18:16)  [GCC 7.5.0]
Pillow  Version  : 9.0.1
PyTorch Version  : 1.12.0
cuDNN   Version  : 8302
CUDA available   : True
CUDA GPU numbers : 2
CUDA_VISIBLE_DEVICES : None


In [4]:
def search_func(xloader, network, criterion, scheduler, w_optimizer, epoch_str, print_freq, logger):
    data_time, batch_time = AverageMeter(), AverageMeter()
    base_losses, base_top1, base_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.train()
    end = time.time()
    for step, (base_inputs, base_targets, arch_inputs, arch_targets) in enumerate(
        xloader
    ):
        scheduler.update(None, 1.0 * step / len(xloader))
        base_targets = base_targets.cuda(non_blocking=True)
        arch_targets = arch_targets.cuda(non_blocking=True)
        # measure data loading time
        data_time.update(time.time() - end)

        # update the weights
        ### sample a cell-wise randomized network
        network.module.random_genotype_per_cell(True)
        ###
        w_optimizer.zero_grad()
        _, logits = network(base_inputs)
        base_loss = criterion(logits, base_targets)
        base_loss.backward()
        nn.utils.clip_grad_norm_(network.parameters(), 5)
        w_optimizer.step()
        # record
        base_prec1, base_prec5 = obtain_accuracy(
            logits.data, base_targets.data, topk=(1, 5)
        )
        base_losses.update(base_loss.item(), base_inputs.size(0))
        base_top1.update(base_prec1.item(), base_inputs.size(0))
        base_top5.update(base_prec5.item(), base_inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if step % print_freq == 0 or step + 1 == len(xloader):
            Sstr = (
                "*SEARCH* "
                + time_string()
                + " [{:}][{:03d}/{:03d}]".format(epoch_str, step, len(xloader))
            )
            Tstr = "Time {batch_time.val:.2f} ({batch_time.avg:.2f}) Data {data_time.val:.2f} ({data_time.avg:.2f})".format(
                batch_time=batch_time, data_time=data_time
            )
            Wstr = "Base [Loss {loss.val:.3f} ({loss.avg:.3f})  Prec@1 {top1.val:.2f} ({top1.avg:.2f}) Prec@5 {top5.val:.2f} ({top5.avg:.2f})]".format(
                loss=base_losses, top1=base_top1, top5=base_top5
            )
            logger.log(Sstr + " " + Tstr + " " + Wstr)
    return base_losses.avg, base_top1.avg, base_top5.avg


def valid_func(xloader, network, criterion):
    data_time, batch_time = AverageMeter(), AverageMeter()
    arch_losses, arch_top1, arch_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.eval()
    end = time.time()
    with torch.no_grad():
        for step, (arch_inputs, arch_targets) in enumerate(xloader):
            arch_targets = arch_targets.cuda(non_blocking=True)
            # measure data loading time
            data_time.update(time.time() - end)
            # prediction

            network.module.random_genotype_per_cell(True)
            _, logits = network(arch_inputs)
            arch_loss = criterion(logits, arch_targets)
            # record
            arch_prec1, arch_prec5 = obtain_accuracy(
                logits.data, arch_targets.data, topk=(1, 5)
            )
            arch_losses.update(arch_loss.item(), arch_inputs.size(0))
            arch_top1.update(arch_prec1.item(), arch_inputs.size(0))
            arch_top5.update(arch_prec5.item(), arch_inputs.size(0))
            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()
    return arch_losses.avg, arch_top1.avg, arch_top5.avg


def search_find_best(xloader, network, n_samples):
    with torch.no_grad():
        network.eval()
        archs, valid_accs = [], []
        # print ('obtain the top-{:} architectures'.format(n_samples))
        loader_iter = iter(xloader)
        for i in range(n_samples):
            arch = network.module.random_genotype_per_cell(True)
            try:
                inputs, targets = next(loader_iter)
            except:
                loader_iter = iter(xloader)
                inputs, targets = next(loader_iter)

            _, logits = network(inputs)
            val_top1, val_top5 = obtain_accuracy(
                logits.cpu().data, targets.data, topk=(1, 5)
            )

            archs.append(arch)
            valid_accs.append(val_top1.item())

        best_idx = np.argmax(valid_accs)
        best_arch, best_valid_acc = archs[best_idx], valid_accs[best_idx]
        return best_arch, best_valid_acc

In [5]:
train_data, valid_data, xshape, class_num = get_datasets(xargs.dataset, xargs.data_path, -1)
config = load_config(xargs.config_path, {"class_num": class_num, "xshape": xshape}, logger)
search_loader, _, valid_loader = get_nas_search_loaders(train_data,
                                                        valid_data,
                                                        xargs.dataset,
                                                        "../configs/nas-benchmark/",
                                                        (config.batch_size, config.test_batch_size),
                                                        xargs.workers)
logger.log("||||||| {:10s} ||||||| Search-Loader-Num={:}, Valid-Loader-Num={:}, batch size={:}".format(
            xargs.dataset, len(search_loader), len(valid_loader), config.batch_size))
logger.log("||||||| {:10s} ||||||| Config={:}".format(xargs.dataset, config))

search_space = get_search_spaces("cell", xargs.search_space_name)
model_config = dict2config(
    {
        "name": "RANDOM",
        "C": xargs.channel,
        "N": xargs.num_cells,
        "max_nodes": xargs.max_nodes,
        "num_classes": class_num,
        "space": search_space,
        "affine": False,
        "track_running_stats": bool(xargs.track_running_stats),
    },
    None,
)
search_model = get_cell_based_tiny_net(model_config)

w_optimizer, w_scheduler, criterion = get_optim_scheduler(search_model.parameters(), config)

logger.log("w-optimizer : {:}".format(w_optimizer))
logger.log("w-scheduler : {:}".format(w_scheduler))
logger.log("criterion   : {:}".format(criterion))
if xargs.arch_nas_dataset is None:
    api = None
else:
    api = API(xargs.arch_nas_dataset)
logger.log("{:} create API = {:} done".format(time_string(), api))

last_info, model_base_path, model_best_path = (
    logger.path("info"),
    logger.path("model"),
    logger.path("best"),
)
network, criterion = torch.nn.DataParallel(search_model).cuda(), criterion.cuda()

if last_info.exists():  # automatically resume from previous checkpoint
    logger.log(
        "=> loading checkpoint of the last-info '{:}' start".format(last_info)
    )
    last_info = torch.load(last_info)
    start_epoch = last_info["epoch"]
    checkpoint = torch.load(last_info["last_checkpoint"])
    genotypes = checkpoint["genotypes"]
    valid_accuracies = checkpoint["valid_accuracies"]
    search_model.load_state_dict(checkpoint["search_model"])
    w_scheduler.load_state_dict(checkpoint["w_scheduler"])
    w_optimizer.load_state_dict(checkpoint["w_optimizer"])
    logger.log(
        "=> loading checkpoint of the last-info '{:}' start with {:}-th epoch.".format(
            last_info, start_epoch
        )
    )
else:
    logger.log("=> do not find the last-info file : {:}".format(last_info))
    start_epoch, valid_accuracies, genotypes = 0, {"best": -1}, {}

Files already downloaded and verified
Files already downloaded and verified
../configs/nas-benchmark/algos/RANDOM.config
Configure(scheduler='cos', LR=0.025, eta_min=0.001, epochs=250, warmup=0, optim='SGD', decay=0.0005, momentum=0.9, nesterov=True, criterion='Softmax', batch_size=64, test_batch_size=512, class_num=10, xshape=(1, 3, 32, 32))
||||||| cifar10    ||||||| Search-Loader-Num=391, Valid-Loader-Num=49, batch size=64
||||||| cifar10    ||||||| Config=Configure(scheduler='cos', LR=0.025, eta_min=0.001, epochs=250, warmup=0, optim='SGD', decay=0.0005, momentum=0.9, nesterov=True, criterion='Softmax', batch_size=64, test_batch_size=512, class_num=10, xshape=(1, 3, 32, 32))
w-optimizer : SGD (
Parameter Group 0
    dampening: 0
    foreach: None
    initial_lr: 0.025
    lr: 0.025
    maximize: False
    momentum: 0.9
    nesterov: True
    weight_decay: 0.0005
)
w-scheduler : CosineAnnealingLR(warmup=0, max-epoch=250, current::epoch=0, iter=0.00, type=cosine, T-max=250, eta-min=0

In [6]:
# start training
start_time, search_time, epoch_time, total_epoch = (
    time.time(),
    AverageMeter(),
    AverageMeter(),
    config.epochs + config.warmup,
)
for epoch in range(start_epoch, total_epoch):
    w_scheduler.update(epoch, 0.0)
    need_time = "Time Left: {:}".format(
        convert_secs2time(epoch_time.val * (total_epoch - epoch), True)
    )
    epoch_str = "{:03d}-{:03d}".format(epoch, total_epoch)
    logger.log(
        "\n[Search the {:}-th epoch] {:}, LR={:}".format(
            epoch_str, need_time, min(w_scheduler.get_lr())
        )
    )

    # selected_arch = search_find_best(valid_loader, network, criterion, xargs.select_num)
    search_w_loss, search_w_top1, search_w_top5 = search_func(
        search_loader,
        network,
        criterion,
        w_scheduler,
        w_optimizer,
        epoch_str,
        xargs.print_freq,
        logger,
    )
    search_time.update(time.time() - start_time)
    logger.log(
        "[{:}] searching : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%, time-cost={:.1f} s".format(
            epoch_str, search_w_loss, search_w_top1, search_w_top5, search_time.sum
        )
    )
    valid_a_loss, valid_a_top1, valid_a_top5 = valid_func(
        valid_loader, network, criterion
    )
    logger.log(
        "[{:}] evaluate  : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%".format(
            epoch_str, valid_a_loss, valid_a_top1, valid_a_top5
        )
    )
    cur_arch, cur_valid_acc = search_find_best(
        valid_loader, network, xargs.select_num
    )
    logger.log(
        "[{:}] find-the-best : {:}, accuracy@1={:.2f}%".format(
            epoch_str, cur_arch, cur_valid_acc
        )
    )
    genotypes[epoch] = cur_arch
    # check the best accuracy
    valid_accuracies[epoch] = valid_a_top1
    if valid_a_top1 > valid_accuracies["best"]:
        valid_accuracies["best"] = valid_a_top1
        find_best = True
    else:
        find_best = False

    # save checkpoint
    save_path = save_checkpoint(
        {
            "epoch": epoch + 1,
            "args": deepcopy(xargs),
            "search_model": search_model.state_dict(),
            "w_optimizer": w_optimizer.state_dict(),
            "w_scheduler": w_scheduler.state_dict(),
            "genotypes": genotypes,
            "valid_accuracies": valid_accuracies,
        },
        model_base_path,
        logger,
    )
    last_info = save_checkpoint(
        {
            "epoch": epoch + 1,
            "args": deepcopy(args),
            "last_checkpoint": save_path,
        },
        logger.path("info"),
        logger,
    )
    if find_best:
        logger.log(
            "<<<--->>> The {:}-th epoch : find the highest validation accuracy : {:.2f}%.".format(
                epoch_str, valid_a_top1
            )
        )
        copy_checkpoint(model_base_path, model_best_path, logger)
    if api is not None:
        logger.log("{:}".format(api.query_by_arch(genotypes[epoch], "200")))
    # measure elapsed time
    epoch_time.update(time.time() - start_time)
    start_time = time.time()

logger.log("\n" + "-" * 200)
logger.log("Pre-searching costs {:.1f} s".format(search_time.sum))
start_time = time.time()
best_arch, best_acc = search_find_best(valid_loader, network, xargs.select_num)
search_time.update(time.time() - start_time)
logger.log(
    "RANDOM-NAS finds the best one : {:} with accuracy={:.2f}%, with {:.1f} s.".format(
        best_arch, best_acc, search_time.sum
    )
)
if api is not None:
    logger.log("{:}".format(api.query_by_arch(best_arch, "200")))
logger.close()


[Search the 000-250-th epoch] Time Left: [00:00:00], LR=0.025
*SEARCH* [2022-09-28 02:13:26] [000-250][000/391] Time 2.30 (2.30) Data 0.16 (0.16) Base [Loss 2.307 (2.307)  Prec@1 6.25 (6.25) Prec@5 53.12 (53.12)]
*SEARCH* [2022-09-28 02:13:28] [000-250][100/391] Time 0.02 (0.04) Data 0.00 (0.00) Base [Loss 2.067 (2.252)  Prec@1 31.25 (14.65) Prec@5 78.12 (59.16)]
*SEARCH* [2022-09-28 02:13:30] [000-250][200/391] Time 0.02 (0.03) Data 0.00 (0.00) Base [Loss 2.121 (2.190)  Prec@1 18.75 (16.78) Prec@5 65.62 (64.70)]
*SEARCH* [2022-09-28 02:13:33] [000-250][300/391] Time 0.03 (0.03) Data 0.00 (0.00) Base [Loss 2.315 (2.144)  Prec@1 10.94 (18.55) Prec@5 51.56 (68.41)]
*SEARCH* [2022-09-28 02:13:35] [000-250][390/391] Time 0.02 (0.03) Data 0.00 (0.00) Base [Loss 1.990 (2.122)  Prec@1 32.50 (19.39) Prec@5 70.00 (69.88)]
[000-250] searching : loss=2.12, accuracy@1=19.39%, accuracy@5=69.88%, time-cost=12.1 s
[000-250] evaluate  : loss=2.12, accuracy@1=21.03%, accuracy@5=75.23%
[000-250] find-t

*SEARCH* [2022-09-28 02:14:32] [004-250][000/391] Time 0.36 (0.36) Data 0.32 (0.32) Base [Loss 1.791 (1.791)  Prec@1 34.38 (34.38) Prec@5 82.81 (82.81)]
*SEARCH* [2022-09-28 02:14:34] [004-250][100/391] Time 0.02 (0.03) Data 0.00 (0.00) Base [Loss 1.554 (1.734)  Prec@1 40.62 (34.75) Prec@5 93.75 (86.65)]
*SEARCH* [2022-09-28 02:14:37] [004-250][200/391] Time 0.02 (0.03) Data 0.00 (0.00) Base [Loss 1.568 (1.723)  Prec@1 32.81 (35.02) Prec@5 92.19 (86.61)]
*SEARCH* [2022-09-28 02:14:39] [004-250][300/391] Time 0.02 (0.03) Data 0.00 (0.00) Base [Loss 1.849 (1.719)  Prec@1 29.69 (35.12) Prec@5 85.94 (86.36)]
*SEARCH* [2022-09-28 02:14:41] [004-250][390/391] Time 0.02 (0.03) Data 0.00 (0.00) Base [Loss 1.561 (1.704)  Prec@1 45.00 (35.82) Prec@5 90.00 (86.86)]
[004-250] searching : loss=1.70, accuracy@1=35.82%, accuracy@5=86.86%, time-cost=55.4 s
[004-250] evaluate  : loss=1.79, accuracy@1=34.52%, accuracy@5=85.44%
[004-250] find-the-best : [Structure(4 nodes with |nor_conv_3x3~0|+|nor_conv_

*SEARCH* [2022-09-28 02:15:40] [008-250][000/391] Time 0.37 (0.37) Data 0.34 (0.34) Base [Loss 1.444 (1.444)  Prec@1 48.44 (48.44) Prec@5 93.75 (93.75)]
*SEARCH* [2022-09-28 02:15:43] [008-250][100/391] Time 0.04 (0.03) Data 0.00 (0.00) Base [Loss 1.496 (1.517)  Prec@1 43.75 (44.69) Prec@5 89.06 (89.76)]
*SEARCH* [2022-09-28 02:15:46] [008-250][200/391] Time 0.03 (0.03) Data 0.00 (0.00) Base [Loss 1.572 (1.506)  Prec@1 45.31 (44.88) Prec@5 93.75 (89.70)]
*SEARCH* [2022-09-28 02:15:49] [008-250][300/391] Time 0.03 (0.03) Data 0.00 (0.00) Base [Loss 1.357 (1.493)  Prec@1 46.88 (45.25) Prec@5 95.31 (90.33)]
*SEARCH* [2022-09-28 02:15:52] [008-250][390/391] Time 0.03 (0.03) Data 0.00 (0.00) Base [Loss 1.369 (1.502)  Prec@1 40.00 (44.75) Prec@5 97.50 (89.86)]
[008-250] searching : loss=1.50, accuracy@1=44.75%, accuracy@5=89.86%, time-cost=103.7 s
[008-250] evaluate  : loss=1.58, accuracy@1=44.11%, accuracy@5=88.84%
[008-250] find-the-best : [Structure(4 nodes with |avg_pool_3x3~0|+|nor_conv

*SEARCH* [2022-09-28 02:16:51] [012-250][100/391] Time 0.02 (0.02) Data 0.00 (0.00) Base [Loss 1.457 (1.350)  Prec@1 42.19 (50.77) Prec@5 92.19 (91.71)]
*SEARCH* [2022-09-28 02:16:54] [012-250][200/391] Time 0.03 (0.03) Data 0.00 (0.00) Base [Loss 1.201 (1.356)  Prec@1 57.81 (51.05) Prec@5 93.75 (91.66)]
*SEARCH* [2022-09-28 02:16:57] [012-250][300/391] Time 0.03 (0.03) Data 0.00 (0.00) Base [Loss 2.349 (1.359)  Prec@1 6.25 (50.72) Prec@5 46.88 (91.06)]
*SEARCH* [2022-09-28 02:17:00] [012-250][390/391] Time 0.03 (0.03) Data 0.00 (0.00) Base [Loss 1.183 (1.347)  Prec@1 60.00 (51.15) Prec@5 95.00 (91.28)]
[012-250] searching : loss=1.35, accuracy@1=51.15%, accuracy@5=91.28%, time-cost=147.9 s
[012-250] evaluate  : loss=1.60, accuracy@1=44.23%, accuracy@5=86.89%
[012-250] find-the-best : [Structure(4 nodes with |avg_pool_3x3~0|+|skip_connect~0|nor_conv_3x3~1|+|skip_connect~0|avg_pool_3x3~1|nor_conv_1x1~2|), Structure(4 nodes with |none~0|+|nor_conv_3x3~0|skip_connect~1|+|skip_connect~0|no

[016-250] find-the-best : [Structure(4 nodes with |skip_connect~0|+|none~0|none~1|+|skip_connect~0|nor_conv_1x1~1|none~2|), Structure(4 nodes with |nor_conv_1x1~0|+|nor_conv_3x3~0|none~1|+|nor_conv_1x1~0|skip_connect~1|nor_conv_1x1~2|), Structure(4 nodes with |nor_conv_1x1~0|+|skip_connect~0|none~1|+|none~0|skip_connect~1|nor_conv_1x1~2|)], accuracy@1=63.92%
Find ../results/220924_single_cell_random/checkpoint/seed-7350-basic.pth exist, delete is at first before saving
save checkpoint into ../results/220924_single_cell_random/checkpoint/seed-7350-basic.pth
Find ../results/220924_single_cell_random/seed-7350-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/220924_single_cell_random/seed-7350-last-info.pth

[Search the 017-250-th epoch] Time Left: [01:06:47], LR=0.024727217482818323
*SEARCH* [2022-09-28 02:18:14] [017-250][000/391] Time 0.31 (0.31) Data 0.29 (0.29) Base [Loss 1.266 (1.266)  Prec@1 57.81 (57.81) Prec@5 96.88 (96.88)]
*SEARCH* [2022-09-

...

*SEARCH* [2022-09-28 03:18:09] [230-250][000/391] Time 0.30 (0.30) Data 0.27 (0.27) Base [Loss 0.248 (0.248)  Prec@1 93.75 (93.75) Prec@5 100.00 (100.00)]
*SEARCH* [2022-09-28 03:18:12] [230-250][100/391] Time 0.02 (0.03) Data 0.00 (0.00) Base [Loss 0.345 (0.556)  Prec@1 85.94 (80.28) Prec@5 100.00 (95.56)]
*SEARCH* [2022-09-28 03:18:14] [230-250][200/391] Time 0.03 (0.03) Data 0.00 (0.00) Base [Loss 2.331 (0.559)  Prec@1 6.25 (80.25) Prec@5 46.88 (95.65)]
*SEARCH* [2022-09-28 03:18:17] [230-250][300/391] Time 0.04 (0.03) Data 0.00 (0.00) Base [Loss 0.540 (0.561)  Prec@1 78.12 (80.08) Prec@5 100.00 (95.75)]
*SEARCH* [2022-09-28 03:18:20] [230-250][390/391] Time 0.02 (0.03) Data 0.00 (0.00) Base [Loss 0.389 (0.568)  Prec@1 85.00 (79.86) Prec@5 100.00 (95.59)]
[230-250] searching : loss=0.57, accuracy@1=79.86%, accuracy@5=95.59%, time-cost=2563.0 s
[230-250] evaluate  : loss=0.85, accuracy@1=72.60%, accuracy@5=96.12%
[230-250] find-the-best : [Structure(4 nodes with |nor_conv_3x3~0|+|non

*SEARCH* [2022-09-28 03:19:31] [235-250][000/391] Time 0.30 (0.30) Data 0.28 (0.28) Base [Loss 0.539 (0.539)  Prec@1 76.56 (76.56) Prec@5 100.00 (100.00)]
*SEARCH* [2022-09-28 03:19:34] [235-250][100/391] Time 0.02 (0.02) Data 0.00 (0.00) Base [Loss 0.618 (0.493)  Prec@1 84.38 (83.03) Prec@5 100.00 (97.49)]
*SEARCH* [2022-09-28 03:19:36] [235-250][200/391] Time 0.02 (0.02) Data 0.00 (0.00) Base [Loss 0.309 (0.499)  Prec@1 90.62 (82.70) Prec@5 100.00 (97.32)]
*SEARCH* [2022-09-28 03:19:38] [235-250][300/391] Time 0.02 (0.02) Data 0.00 (0.00) Base [Loss 0.477 (0.475)  Prec@1 84.38 (83.55) Prec@5 100.00 (97.91)]
*SEARCH* [2022-09-28 03:19:41] [235-250][390/391] Time 0.03 (0.02) Data 0.00 (0.00) Base [Loss 0.496 (0.488)  Prec@1 82.50 (82.96) Prec@5 97.50 (97.76)]
[235-250] searching : loss=0.49, accuracy@1=82.96%, accuracy@5=97.76%, time-cost=2614.6 s
[235-250] evaluate  : loss=1.01, accuracy@1=69.71%, accuracy@5=93.86%
[235-250] find-the-best : [Structure(4 nodes with |none~0|+|avg_pool_3

*SEARCH* [2022-09-28 03:20:51] [240-250][000/391] Time 0.34 (0.34) Data 0.30 (0.30) Base [Loss 0.341 (0.341)  Prec@1 82.81 (82.81) Prec@5 100.00 (100.00)]
*SEARCH* [2022-09-28 03:20:55] [240-250][100/391] Time 0.03 (0.04) Data 0.00 (0.00) Base [Loss 0.414 (0.529)  Prec@1 89.06 (81.50) Prec@5 100.00 (96.49)]
*SEARCH* [2022-09-28 03:20:58] [240-250][200/391] Time 0.03 (0.03) Data 0.00 (0.00) Base [Loss 0.385 (0.509)  Prec@1 89.06 (82.18) Prec@5 98.44 (96.50)]
*SEARCH* [2022-09-28 03:21:01] [240-250][300/391] Time 0.03 (0.03) Data 0.00 (0.00) Base [Loss 0.446 (0.523)  Prec@1 84.38 (81.63) Prec@5 100.00 (96.25)]
*SEARCH* [2022-09-28 03:21:04] [240-250][390/391] Time 0.03 (0.03) Data 0.00 (0.00) Base [Loss 0.375 (0.532)  Prec@1 90.00 (81.22) Prec@5 97.50 (95.99)]
[240-250] searching : loss=0.53, accuracy@1=81.22%, accuracy@5=95.99%, time-cost=2669.3 s
[240-250] evaluate  : loss=0.96, accuracy@1=69.84%, accuracy@5=93.66%
[240-250] find-the-best : [Structure(4 nodes with |skip_connect~0|+|ski

*SEARCH* [2022-09-28 03:22:14] [245-250][000/391] Time 0.30 (0.30) Data 0.26 (0.26) Base [Loss 0.345 (0.345)  Prec@1 87.50 (87.50) Prec@5 100.00 (100.00)]
*SEARCH* [2022-09-28 03:22:17] [245-250][100/391] Time 0.03 (0.04) Data 0.00 (0.00) Base [Loss 0.184 (0.549)  Prec@1 93.75 (80.63) Prec@5 100.00 (96.21)]
*SEARCH* [2022-09-28 03:22:20] [245-250][200/391] Time 0.02 (0.03) Data 0.00 (0.00) Base [Loss 0.277 (0.549)  Prec@1 89.06 (80.50) Prec@5 100.00 (95.91)]
*SEARCH* [2022-09-28 03:22:22] [245-250][300/391] Time 0.02 (0.03) Data 0.00 (0.00) Base [Loss 0.252 (0.558)  Prec@1 92.19 (80.09) Prec@5 100.00 (95.79)]
*SEARCH* [2022-09-28 03:22:24] [245-250][390/391] Time 0.02 (0.03) Data 0.00 (0.00) Base [Loss 0.398 (0.563)  Prec@1 82.50 (79.85) Prec@5 100.00 (95.54)]
[245-250] searching : loss=0.56, accuracy@1=79.85%, accuracy@5=95.54%, time-cost=2719.9 s
[245-250] evaluate  : loss=1.00, accuracy@1=69.65%, accuracy@5=94.44%
[245-250] find-the-best : [Structure(4 nodes with |nor_conv_1x1~0|+|a

RANDOM-NAS finds the best one : [Structure(4 nodes with |skip_connect~0|+|nor_conv_3x3~0|avg_pool_3x3~1|+|avg_pool_3x3~0|nor_conv_1x1~1|nor_conv_3x3~2|), Structure(4 nodes with |none~0|+|avg_pool_3x3~0|skip_connect~1|+|nor_conv_1x1~0|none~1|nor_conv_3x3~2|), Structure(4 nodes with |skip_connect~0|+|none~0|none~1|+|nor_conv_3x3~0|avg_pool_3x3~1|nor_conv_3x3~2|)] with accuracy=86.33%, with 2766.4 s.


# Train a found model

In [7]:
print(best_arch)
import copy
best_arch_copy = copy.deepcopy(best_arch)

[Structure(4 nodes with |skip_connect~0|+|nor_conv_3x3~0|avg_pool_3x3~1|+|avg_pool_3x3~0|nor_conv_1x1~1|nor_conv_3x3~2|), Structure(4 nodes with |none~0|+|avg_pool_3x3~0|skip_connect~1|+|nor_conv_1x1~0|none~1|nor_conv_3x3~2|), Structure(4 nodes with |skip_connect~0|+|none~0|none~1|+|nor_conv_3x3~0|avg_pool_3x3~1|nor_conv_3x3~2|)]


In [8]:
print(args)
tmp = os.path.join(args.save_dir, "train_best_arch")
args.save_dir = tmp
print(args)

Namespace(arch_nas_dataset=None, channel=16, config_path='../configs/nas-benchmark/algos/RANDOM.config', data_path='../cifar.python', dataset='cifar10', max_nodes=4, num_cells=1, print_freq=100, rand_seed=7350, save_dir='../results/220924_single_cell_random/', search_space_name='nas-bench-201', select_num=100, track_running_stats=0, workers=4)
Namespace(arch_nas_dataset=None, channel=16, config_path='../configs/nas-benchmark/algos/RANDOM.config', data_path='../cifar.python', dataset='cifar10', max_nodes=4, num_cells=1, print_freq=100, rand_seed=7350, save_dir='../results/220924_single_cell_random/train_best_arch', search_space_name='nas-bench-201', select_num=100, track_running_stats=0, workers=4)


In [9]:
logger = prepare_logger(args)

cifar_train_config_path = "../configs/nas-benchmark/CIFAR.config"
###
train_data, valid_data, xshape, class_num = get_datasets(xargs.dataset, xargs.data_path, -1)
config = load_config(cifar_train_config_path, {"class_num": class_num, "xshape": xshape}, logger)
search_loader, _, valid_loader = get_nas_search_loaders(train_data,
                                                        valid_data,
                                                        xargs.dataset,
                                                        "../configs/nas-benchmark/",
                                                        (config.batch_size, config.batch_size),
                                                        xargs.workers)
logger.log("||||||| {:10s} ||||||| Search-Loader-Num={:}, Valid-Loader-Num={:}, batch size={:}".format(
            xargs.dataset, len(search_loader), len(valid_loader), config.batch_size))
logger.log("||||||| {:10s} ||||||| Config={:}".format(xargs.dataset, config))

search_space = get_search_spaces("cell", xargs.search_space_name)
model_config = dict2config(
    {
        "name": "RANDOM",
        "C": xargs.channel,
        "N": xargs.num_cells,
        "max_nodes": xargs.max_nodes,
        "num_classes": class_num,
        "space": search_space,
        "affine": False,
        "track_running_stats": True, # true for eval
    },
    None,
)
search_model = get_cell_based_tiny_net(model_config)

w_optimizer, w_scheduler, criterion = get_optim_scheduler(search_model.parameters(), config)

logger.log("w-optimizer : {:}".format(w_optimizer))
logger.log("w-scheduler : {:}".format(w_scheduler))
logger.log("criterion   : {:}".format(criterion))

network, criterion = torch.nn.DataParallel(search_model).cuda(), criterion.cuda()

last_info, model_base_path, model_best_path = (
    logger.path("info"),
    logger.path("model"),
    logger.path("best"),
)

start_epoch, valid_accuracies, genotypes = 0, {"best": -1}, {}

Main Function with logger : Logger(dir=../results/220924_single_cell_random/train_best_arch, use-tf=False, writer=None)
Arguments : -------------------------------
arch_nas_dataset : None
channel          : 16
config_path      : ../configs/nas-benchmark/algos/RANDOM.config
data_path        : ../cifar.python
dataset          : cifar10
max_nodes        : 4
num_cells        : 1
print_freq       : 100
rand_seed        : 7350
save_dir         : ../results/220924_single_cell_random/train_best_arch
search_space_name : nas-bench-201
select_num       : 100
track_running_stats : 0
workers          : 4
Python  Version  : 3.7.13 (default, Mar 29 2022, 02:18:16)  [GCC 7.5.0]
Pillow  Version  : 9.0.1
PyTorch Version  : 1.12.0
cuDNN   Version  : 8302
CUDA available   : True
CUDA GPU numbers : 2
CUDA_VISIBLE_DEVICES : None
Files already downloaded and verified
Files already downloaded and verified
../configs/nas-benchmark/CIFAR.config
Configure(scheduler='cos', eta_min=0.0, epochs=200, warmup=0, optim

In [10]:
from custom_search_cells import NAS201SearchCell as SearchCell

i = 0
for cell in network.module.cells:
    if isinstance(cell, SearchCell):
        cell.arch_cache = best_arch[i]
        i = i+1
print(i)
print(len(best_arch))

3
3


In [11]:
def search_func_one_arch(xloader, network, criterion, scheduler, w_optimizer, epoch_str, print_freq, logger):
    data_time, batch_time = AverageMeter(), AverageMeter()
    base_losses, base_top1, base_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.train()
    end = time.time()
    for step, (base_inputs, base_targets, arch_inputs, arch_targets) in enumerate(
        xloader
    ):
        scheduler.update(None, 1.0 * step / len(xloader))
        base_targets = base_targets.cuda(non_blocking=True)
        arch_targets = arch_targets.cuda(non_blocking=True)
        # measure data loading time
        data_time.update(time.time() - end)

        # update the weights
        ### sample a cell-wise randomized network
#         network.module.random_genotype_per_cell(True)
        ###
        w_optimizer.zero_grad()
        _, logits = network(base_inputs)
        base_loss = criterion(logits, base_targets)
        base_loss.backward()
        nn.utils.clip_grad_norm_(network.parameters(), 5)
        w_optimizer.step()
        # record
        base_prec1, base_prec5 = obtain_accuracy(
            logits.data, base_targets.data, topk=(1, 5)
        )
        base_losses.update(base_loss.item(), base_inputs.size(0))
        base_top1.update(base_prec1.item(), base_inputs.size(0))
        base_top5.update(base_prec5.item(), base_inputs.size(0))

        # measure elapsed time
        batch_time.update(time.time() - end)
        end = time.time()

        if step % print_freq == 0 or step + 1 == len(xloader):
            Sstr = (
                "*SEARCH* "
                + time_string()
                + " [{:}][{:03d}/{:03d}]".format(epoch_str, step, len(xloader))
            )
            Tstr = "Time {batch_time.val:.2f} ({batch_time.avg:.2f}) Data {data_time.val:.2f} ({data_time.avg:.2f})".format(
                batch_time=batch_time, data_time=data_time
            )
            Wstr = "Base [Loss {loss.val:.3f} ({loss.avg:.3f})  Prec@1 {top1.val:.2f} ({top1.avg:.2f}) Prec@5 {top5.val:.2f} ({top5.avg:.2f})]".format(
                loss=base_losses, top1=base_top1, top5=base_top5
            )
            logger.log(Sstr + " " + Tstr + " " + Wstr)
    return base_losses.avg, base_top1.avg, base_top5.avg

def valid_func_one_arch(xloader, network, criterion):
    data_time, batch_time = AverageMeter(), AverageMeter()
    arch_losses, arch_top1, arch_top5 = AverageMeter(), AverageMeter(), AverageMeter()
    network.eval()
    end = time.time()
    with torch.no_grad():
        for step, (arch_inputs, arch_targets) in enumerate(xloader):
            arch_targets = arch_targets.cuda(non_blocking=True)
            # measure data loading time
            data_time.update(time.time() - end)
            # prediction

#             network.module.random_genotype_per_cell(True)
            _, logits = network(arch_inputs)
            arch_loss = criterion(logits, arch_targets)
            # record
            arch_prec1, arch_prec5 = obtain_accuracy(
                logits.data, arch_targets.data, topk=(1, 5)
            )
            arch_losses.update(arch_loss.item(), arch_inputs.size(0))
            arch_top1.update(arch_prec1.item(), arch_inputs.size(0))
            arch_top5.update(arch_prec5.item(), arch_inputs.size(0))
            # measure elapsed time
            batch_time.update(time.time() - end)
            end = time.time()
    return arch_losses.avg, arch_top1.avg, arch_top5.avg

In [12]:
start_time, search_time, epoch_time, total_epoch = (
    time.time(),
    AverageMeter(),
    AverageMeter(),
    config.epochs + config.warmup,
)
for epoch in range(0, total_epoch):
    w_scheduler.update(epoch, 0.0)
    need_time = "Time Left: {:}".format(
        convert_secs2time(epoch_time.val * (total_epoch - epoch), True)
    )
    epoch_str = "{:03d}-{:03d}".format(epoch, total_epoch)
    logger.log(
        "\n[Search the {:}-th epoch] {:}, LR={:}".format(
            epoch_str, need_time, min(w_scheduler.get_lr())
        )
    )

    # selected_arch = search_find_best(valid_loader, network, criterion, xargs.select_num)
    search_w_loss, search_w_top1, search_w_top5 = search_func_one_arch(
        search_loader,
        network,
        criterion,
        w_scheduler,
        w_optimizer,
        epoch_str,
        xargs.print_freq,
        logger,
    )
    search_time.update(time.time() - start_time)
    logger.log(
        "[{:}] searching : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%, time-cost={:.1f} s".format(
            epoch_str, search_w_loss, search_w_top1, search_w_top5, search_time.sum
        )
    )
    valid_a_loss, valid_a_top1, valid_a_top5 = valid_func_one_arch(
        valid_loader, network, criterion
    )
    logger.log(
        "[{:}] evaluate  : loss={:.2f}, accuracy@1={:.2f}%, accuracy@5={:.2f}%".format(
            epoch_str, valid_a_loss, valid_a_top1, valid_a_top5
        )
    )
    
    # check the best accuracy
    valid_accuracies[epoch] = valid_a_top1
    if valid_a_top1 > valid_accuracies["best"]:
        valid_accuracies["best"] = valid_a_top1
        find_best = True
    else:
        find_best = False

    # save checkpoint
    save_path = save_checkpoint(
        {
            "epoch": epoch + 1,
            "args": deepcopy(xargs),
            "search_model": search_model.state_dict(),
            "w_optimizer": w_optimizer.state_dict(),
            "w_scheduler": w_scheduler.state_dict(),
            "genotypes": genotypes,
            "valid_accuracies": valid_accuracies,
        },
        model_base_path,
        logger,
    )
    last_info = save_checkpoint(
        {
            "epoch": epoch + 1,
            "args": deepcopy(args),
            "last_checkpoint": save_path,
        },
        logger.path("info"),
        logger,
    )
    if find_best:
        logger.log(
            "<<<--->>> The {:}-th epoch : find the highest validation accuracy : {:.2f}%.".format(
                epoch_str, valid_a_top1
            )
        )
        copy_checkpoint(model_base_path, model_best_path, logger)
    if api is not None:
        logger.log("{:}".format(api.query_by_arch(genotypes[epoch], "200")))
    # measure elapsed time
    epoch_time.update(time.time() - start_time)
    start_time = time.time()

logger.close()


[Search the 000-200-th epoch] Time Left: [00:00:00], LR=0.1
*SEARCH* [2022-09-28 03:23:41] [000-200][000/098] Time 0.40 (0.40) Data 0.35 (0.35) Base [Loss 2.325 (2.325)  Prec@1 8.98 (8.98) Prec@5 48.44 (48.44)]
*SEARCH* [2022-09-28 03:23:45] [000-200][097/098] Time 0.03 (0.05) Data 0.00 (0.01) Base [Loss 1.698 (1.713)  Prec@1 36.31 (35.56) Prec@5 84.52 (87.01)]
[000-200] searching : loss=1.71, accuracy@1=35.56%, accuracy@5=87.01%, time-cost=4.5 s
[000-200] evaluate  : loss=1.84, accuracy@1=35.23%, accuracy@5=85.91%
save checkpoint into ../results/220924_single_cell_random/train_best_arch/checkpoint/seed-7350-basic.pth
save checkpoint into ../results/220924_single_cell_random/train_best_arch/seed-7350-last-info.pth
<<<--->>> The 000-200-th epoch : find the highest validation accuracy : 35.23%.
copy the file from ../results/220924_single_cell_random/train_best_arch/checkpoint/seed-7350-basic.pth into ../results/220924_single_cell_random/train_best_arch/checkpoint/seed-7350-best.pth

[Se

*SEARCH* [2022-09-28 03:24:28] [007-200][000/098] Time 0.36 (0.36) Data 0.32 (0.32) Base [Loss 0.867 (0.867)  Prec@1 67.58 (67.58) Prec@5 97.27 (97.27)]
*SEARCH* [2022-09-28 03:24:32] [007-200][097/098] Time 0.03 (0.04) Data 0.00 (0.01) Base [Loss 0.790 (0.776)  Prec@1 73.21 (72.16) Prec@5 97.62 (98.20)]
[007-200] searching : loss=0.78, accuracy@1=72.16%, accuracy@5=98.20%, time-cost=36.4 s
[007-200] evaluate  : loss=0.87, accuracy@1=69.95%, accuracy@5=97.37%
Find ../results/220924_single_cell_random/train_best_arch/checkpoint/seed-7350-basic.pth exist, delete is at first before saving
save checkpoint into ../results/220924_single_cell_random/train_best_arch/checkpoint/seed-7350-basic.pth
Find ../results/220924_single_cell_random/train_best_arch/seed-7350-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/220924_single_cell_random/train_best_arch/seed-7350-last-info.pth
<<<--->>> The 007-200-th epoch : find the highest validation accuracy : 69.95%.
Fi

*SEARCH* [2022-09-28 03:25:14] [014-200][000/098] Time 0.39 (0.39) Data 0.34 (0.34) Base [Loss 0.497 (0.497)  Prec@1 81.25 (81.25) Prec@5 100.00 (100.00)]
*SEARCH* [2022-09-28 03:25:18] [014-200][097/098] Time 0.02 (0.05) Data 0.00 (0.01) Base [Loss 0.563 (0.598)  Prec@1 82.14 (79.01) Prec@5 99.40 (98.96)]
[014-200] searching : loss=0.60, accuracy@1=79.01%, accuracy@5=98.96%, time-cost=68.2 s
[014-200] evaluate  : loss=0.95, accuracy@1=69.53%, accuracy@5=97.95%
Find ../results/220924_single_cell_random/train_best_arch/checkpoint/seed-7350-basic.pth exist, delete is at first before saving
save checkpoint into ../results/220924_single_cell_random/train_best_arch/checkpoint/seed-7350-basic.pth
Find ../results/220924_single_cell_random/train_best_arch/seed-7350-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/220924_single_cell_random/train_best_arch/seed-7350-last-info.pth

[Search the 015-200-th epoch] Time Left: [00:20:09], LR=0.09861849601988383
*SE

*SEARCH* [2022-09-28 03:26:05] [022-200][000/098] Time 0.35 (0.35) Data 0.31 (0.31) Base [Loss 0.446 (0.446)  Prec@1 83.20 (83.20) Prec@5 100.00 (100.00)]
*SEARCH* [2022-09-28 03:26:09] [022-200][097/098] Time 0.03 (0.05) Data 0.00 (0.02) Base [Loss 0.576 (0.524)  Prec@1 79.76 (81.46) Prec@5 99.40 (99.16)]
[022-200] searching : loss=0.52, accuracy@1=81.46%, accuracy@5=99.16%, time-cost=104.3 s
[022-200] evaluate  : loss=0.72, accuracy@1=76.15%, accuracy@5=98.36%
Find ../results/220924_single_cell_random/train_best_arch/checkpoint/seed-7350-basic.pth exist, delete is at first before saving
save checkpoint into ../results/220924_single_cell_random/train_best_arch/checkpoint/seed-7350-basic.pth
Find ../results/220924_single_cell_random/train_best_arch/seed-7350-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/220924_single_cell_random/train_best_arch/seed-7350-last-info.pth
<<<--->>> The 022-200-th epoch : find the highest validation accuracy : 76.15%.

*SEARCH* [2022-09-28 03:27:00] [030-200][000/098] Time 0.35 (0.35) Data 0.32 (0.32) Base [Loss 0.409 (0.409)  Prec@1 86.72 (86.72) Prec@5 100.00 (100.00)]
*SEARCH* [2022-09-28 03:27:04] [030-200][097/098] Time 0.03 (0.04) Data 0.00 (0.01) Base [Loss 0.413 (0.484)  Prec@1 86.31 (83.17) Prec@5 100.00 (99.38)]
[030-200] searching : loss=0.48, accuracy@1=83.17%, accuracy@5=99.38%, time-cost=140.6 s
[030-200] evaluate  : loss=0.86, accuracy@1=73.46%, accuracy@5=98.12%
Find ../results/220924_single_cell_random/train_best_arch/checkpoint/seed-7350-basic.pth exist, delete is at first before saving
save checkpoint into ../results/220924_single_cell_random/train_best_arch/checkpoint/seed-7350-basic.pth
Find ../results/220924_single_cell_random/train_best_arch/seed-7350-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/220924_single_cell_random/train_best_arch/seed-7350-last-info.pth

[Search the 031-200-th epoch] Time Left: [00:17:36], LR=0.09418828150443469
*

...

[167-200] evaluate  : loss=0.49, accuracy@1=86.28%, accuracy@5=99.28%
Find ../results/220924_single_cell_random/train_best_arch/checkpoint/seed-7350-basic.pth exist, delete is at first before saving
save checkpoint into ../results/220924_single_cell_random/train_best_arch/checkpoint/seed-7350-basic.pth
Find ../results/220924_single_cell_random/train_best_arch/seed-7350-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/220924_single_cell_random/train_best_arch/seed-7350-last-info.pth

[Search the 168-200-th epoch] Time Left: [00:03:42], LR=0.006184665997806821
*SEARCH* [2022-09-28 03:42:25] [168-200][000/098] Time 0.39 (0.39) Data 0.35 (0.35) Base [Loss 0.079 (0.079)  Prec@1 98.05 (98.05) Prec@5 100.00 (100.00)]
*SEARCH* [2022-09-28 03:42:29] [168-200][097/098] Time 0.03 (0.04) Data 0.00 (0.01) Base [Loss 0.109 (0.089)  Prec@1 97.02 (97.28) Prec@5 100.00 (100.00)]
[168-200] searching : loss=0.09, accuracy@1=97.28%, accuracy@5=100.00%, time-cost=770.9 

[174-200] evaluate  : loss=0.48, accuracy@1=86.90%, accuracy@5=99.34%
Find ../results/220924_single_cell_random/train_best_arch/checkpoint/seed-7350-basic.pth exist, delete is at first before saving
save checkpoint into ../results/220924_single_cell_random/train_best_arch/checkpoint/seed-7350-basic.pth
Find ../results/220924_single_cell_random/train_best_arch/seed-7350-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/220924_single_cell_random/train_best_arch/seed-7350-last-info.pth
<<<--->>> The 174-200-th epoch : find the highest validation accuracy : 86.90%.
Find ../results/220924_single_cell_random/train_best_arch/checkpoint/seed-7350-best.pth exist, delete is at first before saving
copy the file from ../results/220924_single_cell_random/train_best_arch/checkpoint/seed-7350-basic.pth into ../results/220924_single_cell_random/train_best_arch/checkpoint/seed-7350-best.pth

[Search the 175-200-th epoch] Time Left: [00:02:50], LR=0.003806023374435674

[181-200] evaluate  : loss=0.48, accuracy@1=87.14%, accuracy@5=99.30%
Find ../results/220924_single_cell_random/train_best_arch/checkpoint/seed-7350-basic.pth exist, delete is at first before saving
save checkpoint into ../results/220924_single_cell_random/train_best_arch/checkpoint/seed-7350-basic.pth
Find ../results/220924_single_cell_random/train_best_arch/seed-7350-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/220924_single_cell_random/train_best_arch/seed-7350-last-info.pth
<<<--->>> The 181-200-th epoch : find the highest validation accuracy : 87.14%.
Find ../results/220924_single_cell_random/train_best_arch/checkpoint/seed-7350-best.pth exist, delete is at first before saving
copy the file from ../results/220924_single_cell_random/train_best_arch/checkpoint/seed-7350-basic.pth into ../results/220924_single_cell_random/train_best_arch/checkpoint/seed-7350-best.pth

[Search the 182-200-th epoch] Time Left: [00:01:56], LR=0.001985315716152852

*SEARCH* [2022-09-28 03:44:46] [189-200][000/098] Time 0.39 (0.39) Data 0.36 (0.36) Base [Loss 0.074 (0.074)  Prec@1 97.66 (97.66) Prec@5 100.00 (100.00)]
*SEARCH* [2022-09-28 03:44:50] [189-200][097/098] Time 0.03 (0.05) Data 0.00 (0.02) Base [Loss 0.083 (0.059)  Prec@1 98.81 (98.50) Prec@5 100.00 (100.00)]
[189-200] searching : loss=0.06, accuracy@1=98.50%, accuracy@5=100.00%, time-cost=867.1 s
[189-200] evaluate  : loss=0.48, accuracy@1=87.35%, accuracy@5=99.30%
Find ../results/220924_single_cell_random/train_best_arch/checkpoint/seed-7350-basic.pth exist, delete is at first before saving
save checkpoint into ../results/220924_single_cell_random/train_best_arch/checkpoint/seed-7350-basic.pth
Find ../results/220924_single_cell_random/train_best_arch/seed-7350-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/220924_single_cell_random/train_best_arch/seed-7350-last-info.pth
<<<--->>> The 189-200-th epoch : find the highest validation accuracy : 87.3

*SEARCH* [2022-09-28 03:45:32] [196-200][000/098] Time 0.52 (0.52) Data 0.48 (0.48) Base [Loss 0.038 (0.038)  Prec@1 99.61 (99.61) Prec@5 100.00 (100.00)]
*SEARCH* [2022-09-28 03:45:36] [196-200][097/098] Time 0.02 (0.05) Data 0.00 (0.02) Base [Loss 0.055 (0.056)  Prec@1 98.81 (98.60) Prec@5 100.00 (99.99)]
[196-200] searching : loss=0.06, accuracy@1=98.60%, accuracy@5=99.99%, time-cost=899.0 s
[196-200] evaluate  : loss=0.48, accuracy@1=87.32%, accuracy@5=99.32%
Find ../results/220924_single_cell_random/train_best_arch/checkpoint/seed-7350-basic.pth exist, delete is at first before saving
save checkpoint into ../results/220924_single_cell_random/train_best_arch/checkpoint/seed-7350-basic.pth
Find ../results/220924_single_cell_random/train_best_arch/seed-7350-last-info.pth exist, delete is at first before saving
save checkpoint into ../results/220924_single_cell_random/train_best_arch/seed-7350-last-info.pth

[Search the 197-200-th epoch] Time Left: [00:00:19], LR=5.550625190150482e-05